In [1]:
import librosa
import os
from IPython.display import Audio
import soundfile as sf

In [11]:
number = '0239'

# input_dir = '/home/cyan/workstation/DL_music/final/audiocraft/examples/cond_finetune_from_mid/'
# input_dir = '/home/cyan/workstation/DL_music/final/audiocraft/examples/cond_finetune/'
input_dir = '/home/cyan/workstation/DL_music/final/audiocraft/examples/gt_accom/'

vocal_dir = '/home/cyan/workstation/DL_music/final/audiocraft/examples/vocal_lead/'

In [14]:
start = 57

dur = 16
accom, sr = librosa.load(os.path.join(input_dir, f"{number}.wav"), sr=32000)
vocal, sr = librosa.load(os.path.join(vocal_dir, f"{number}.wav"), sr=32000)
vocal = vocal[int(sr*start):int(sr*(dur+start))]
accom = accom[int(sr*start):int(sr*(dur+start))]

sf.write(f'./examples/merge/{number}.wav', accom*0.5+vocal, sr)

In [18]:
import torchaudio
import torch
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write
from glob import glob

start = 1.8
number = '0231'
# model = MusicGen.get_pretrained('ckpt/cond_finetune')
model = MusicGen.get_pretrained('ckpt/cond_finetune_from_mid')
dur = 16
model.set_generation_params(duration=16, extend_stride=8)  # generate 8 seconds.

file = f"/home/cyan/workstation/DL_music/final/audiocraft/examples/vocal_lead/{number}.wav"
# file = f"/home/cyan/workstation/DL_music/final/audiocraft/dataset/acca_32k/vocal/{number}.wav"
accom_f = f"/home/cyan/workstation/DL_music/final/audiocraft/examples/gt_accom/{number}.wav"

vocal, sr = librosa.load(file, sr=32000)
vocal = vocal[int(sr*start):int(sr*(dur+start))]

melody, sr = torchaudio.load(file)
melody = torchaudio.functional.resample(melody, orig_freq=sr, new_freq=22050)
melody = melody[:, int(22050*start):int(22050*(dur+start))]
# melody = torchaudio.functional.resample(melody, orig_freq=32000, new_freq=22050)


prompt, sr = torchaudio.load(accom_f)
prompt = torchaudio.functional.resample(prompt, orig_freq=sr, new_freq=32000)
prompt = prompt[:, int(32000*start):int(32000*(4+start))]

descriptions = [''] * 16
# wav = model.generate_with_vocal_lead(descriptions, melody[None].expand(16, -1, -1), sr)  # generates 3 samples.
wav = model.generate_with_vocal_lead_and_prompt(descriptions, melody[None].expand(16, -1, -1), prompt[None].expand(16, -1, -1), 22050, 32000)

for idx, one_wav in enumerate(wav):
    audio_write(f'examples/cherry_pick/{idx}', one_wav.cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)
    audio_write(f'examples/cherry_pick/merge_{idx}', one_wav.cpu()*0.5 + vocal[None], model.sample_rate, strategy="loudness", loudness_compressor=True)



Some weights of the model checkpoint at m-a-p/MERT-v1-330M were not used when initializing MERTModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing MERTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MERTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MERTModel were not initialized from the model checkpoint at m-a-p/MERT-v1-330M and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
